# Using the TFLite Converter

> In this post, we will use TFLite Converter. This is the summary of lecture "Applications of TinyML" from edX.

- toc: true 
- badges: true
- comments: true
- author: Chanseok Kang
- categories: [Python, edX, Deep_Learning, Tensorflow, tinyML]
- image: 

In [10]:
import tensorflow as tf
import numpy as np
import pathlib
from pprint import pprint

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

print("Tensorflow : v" + tf.__version__)

Tensorflow : v2.3.1


## model training

In [3]:
IO = Dense(units=1, input_shape=[1])
model = Sequential([
    IO
])
model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs=500)

print(model.predict([10.0]))
print('Here is what I learned: {}'.format(IO.get_weights()))

Epoch 1/500
1/1 [==============================] - 0s 1ms/step - loss: 50.3373
Epoch 2/500
1/1 [==============================] - 0s 1ms/step - loss: 39.9983
Epoch 3/500
1/1 [==============================] - 0s 1ms/step - loss: 31.8560
Epoch 4/500
1/1 [==============================] - 0s 0s/step - loss: 25.4420
Epoch 5/500
1/1 [==============================] - 0s 1ms/step - loss: 20.3880
Epoch 6/500
1/1 [==============================] - 0s 1ms/step - loss: 16.4042
Epoch 7/500
1/1 [==============================] - 0s 1ms/step - loss: 13.2624
Epoch 8/500
1/1 [==============================] - 0s 1ms/step - loss: 10.7833
Epoch 9/500
1/1 [==============================] - 0s 1000us/step - loss: 8.8257
Epoch 10/500
1/1 [==============================] - 0s 1ms/step - loss: 7.2785
Epoch 11/500
1/1 [==============================] - 0s 0s/step - loss: 6.0543
Epoch 12/500
1/1 [==============================] - 0s 1ms/step - loss: 5.0845
Epoch 13/500
1/1 [==============================] - 

## Model save

In [5]:
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/1\assets


## Model Conversion

In [7]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

In [8]:
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

776

## Load & Inference from model

In [11]:
# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

pprint(input_details)
pprint(output_details)

[{'dtype': <class 'numpy.float32'>,
  'index': 0,
  'name': 'dense_1_input',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
                              'scales': array([], dtype=float32),
                              'zero_points': array([], dtype=int32)},
  'shape': array([1, 1]),
  'shape_signature': array([-1,  1]),
  'sparsity_parameters': {}}]
[{'dtype': <class 'numpy.float32'>,
  'index': 3,
  'name': 'Identity',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
                              'scales': array([], dtype=float32),
                              'zero_points': array([], dtype=int32)},
  'shape': array([1, 1]),
  'shape_signature': array([-1,  1]),
  'sparsity_parameters': {}}]


In [12]:
to_predict = np.array([[10.0]], dtype=np.float32)
print(to_predict)

[[10.]]


In [13]:
interpreter.set_tensor(input_details[0]['index'], to_predict)
interpreter.invoke()

tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(tflite_results)

[[18.97644]]
